In [2]:
import numpy as np
import pandas as pd

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

Нулевая гипотеза: $H_0: \mu_{train} = \mu_{test}$

In [4]:
n = 50
n_zev = 34
n_ne_zev = 16
n_zev_zev = 10
n_ne_zev_zev = 4

In [5]:
p1 = n_zev_zev/n_zev
p2 = n_ne_zev_zev/n_ne_zev
P = float(p1*n_zev + p2*n_ne_zev) / (n_zev + n_ne_zev)

z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n_zev + 1. / n_ne_zev))

In [17]:
import scipy.stats

In [16]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

Здесь p1 и p2 перепутаны местами, поэтому нам нужно выбрать алтернативу "greater", not "less"

In [23]:
p_value = proportions_diff_z_test(z_stat, 'greater')

In [24]:
p_value

0.37293045872523534

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X_1,X_2,X_3X 
1
​	
 ,X 
2
​	
 ,X 
3
​	
 ;
логистическая регрессия по признакам X_4,X_5,X_6X 
4
​	
 ,X 
5
​	
 ,X 
6
​	
 .
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 
−8
 , нужно ввести 8).

In [1]:
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('banknotes.txt', delimiter='\t')

In [4]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [5]:
x = data.drop('real', axis=1)
y = data.real

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

In [7]:
from sklearn.linear_model import LogisticRegression

In [12]:
x_train.iloc[:, 0:3]

,X1,X2,X3
98,215.1,130.0,129.8
123,215.1,130.7,130.4
119,214.4,130.2,129.9
53,215.4,130.2,130.2
33,215.6,130.4,130.1
...,...,...,...
133,214.6,130.2,130.4
137,214.9,130.7,130.3
72,215.0,129.6,130.2
140,214.8,130.2,130.3


In [26]:
model_123 = LogisticRegression(solver='liblinear')
model_456 = LogisticRegression(solver='liblinear')
model_123.fit(x_train.iloc[:,0:3], y_train)
model_456.fit(x_train.iloc[:,3:6], y_train)
pred_123 = model_123.predict(x_test.iloc[:,0:3])
pred_456 = model_456.predict(x_test.iloc[:,3:6])

In [13]:
from sklearn.metrics import accuracy_score

In [27]:
score_123 = accuracy_score(y_test, pred_123)
score_456 = accuracy_score(y_test, pred_456)
print(f'Score_123: {score_123}')
print(f'Score_456: {score_456}')

Score_123: 0.8
Score_456: 0.98


Предсказания сильно разные, но нужно найти достигаемый уровень значимости:

In [28]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [29]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [23]:
import scipy.stats

Так как выборки связанные:

In [30]:
sample1 = abs(pred_123 - y_test)
sample2 = abs(pred_456 - y_test)
z_stat = proportions_diff_z_stat_rel(sample1, sample2)
p_val = proportions_diff_z_test(z_stat)

In [31]:
p_val

0.0032969384555543435

In [32]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [33]:
proportions_diff_confint_rel(sample1, sample2)

(0.059945206279614305, 0.3000547937203857)

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

Используем одновыборочный z-критерий:

In [35]:
mu_mean = 541.4
mu_0 = 525
sigma = 100
n = 100

In [38]:
z = (mu_mean - mu_0)*np.sqrt(n)/sigma

In [39]:
z

1.6399999999999977

$H_0: \mu_1 < \mu_0$, $H_1: \mu_1 > \mu_0$

In [40]:
p = 1 - scipy.stats.norm.cdf(z)

In [41]:
p

0.05050258347410397

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [42]:
mu_mean = 541.5

In [43]:
z = (mu_mean - mu_0)*np.sqrt(n)/sigma

In [44]:
p = 1 - scipy.stats.norm.cdf(z)

In [45]:
p

0.0494714680336481